In [ ]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Create Transformers pipeline for inferencing
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                )

KeyboardInterrupt: ignored

In [ ]:
# Create Transformers pipeline for inferencing on single GPU
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                  device_map="cuda:0"
                                )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
# Create Transformers pipeline for inferencing on both CPU & GPU
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                  device_map="auto"
                                )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 17.7 s, sys: 15.7 s, total: 33.4 s
Wall time: 15.6 s


In [ ]:
prompt = 'Tell me about AI'

In [ ]:
%%time
sequences = pipeline(
    prompt,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: Tell me about AI in healthcare and its potential impact on the industry.
AI in healthcare is a rapidly growing field that has the potential to transform the industry in several ways. Here are some of the ways AI is being used in healthcare and its potential impact:
1. Predictive analytics: AI algorithms can analyze large amounts of data to identify patterns and predict patient outcomes, such as the likelihood of disease progression or response to treatment. This can help healthcare providers make more informed decisions and improve patient care.
2. Medical imaging: AI can be used to analyze medical images such as X-rays and MRIs to identify abnormalities and diagnose conditions. This can help doctors to make more accurate diagnoses and improve treatment outcomes.
3. Personalized medicine: AI can help personalize treatment plans for individual patients based on their unique genetic profiles and medical histories. This can lead
CPU times: user 10min 56s, sys: 3.25 s, total: 10min